In [1]:
import json
import numpy as np
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


def get_squad2():
    import datasets
    from datasets import load_dataset
    dataset = load_dataset('squad_v2', split='validation')
    gold = {}
    idk_gold = []
    for d in dataset:
        gold[d['id']] = d['answers']
        if not d['answers']['text']:
            idk_gold.append(d['id'])
    open('idk_ids.txt', 'w').write('\n'.join(idk_gold))
    return gold

In [256]:
import datasets
from datasets import load_dataset
def featurization(split):
    dataset = load_dataset('squad_v2', split=split)
    features = []
    for d in dataset:
        answers = set()
        if len(d['answers']['text']) > 0:
            for i in range(len(d['answers']['text'])):
                length = len(tokenizer.encode(d['answers']['text'][i]))
                start = d['answers']['answer_start'][i]
                answers.add((start, start + length))
        feature = {'prompt': tokenizer.encode(d['context'] + "\n" + d['question']), \
                   'answers': answers
                  }
        features.append(feature)
    return features
train_dataloader = DataLoader(featurization('train'), batch_size=1, shuffle=True, drop_last=True)
valid_dataloader = DataLoader(featurization('validation'), batch_size=1, shuffle=False, drop_last=True)


Reusing dataset squad_v2 (/home1/w/why16gzl/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


In [257]:
features[0]

{'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'In what country is Normandy located?',
 'answers': {(159, 160)}}

In [3]:
len(get_squad2())

Reusing dataset squad_v2 (/home1/w/why16gzl/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


11873

In [4]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "allenai/unifiedqa-t5-small" # you can specify the model size here
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    return tokenizer.batch_decode(res, skip_special_tokens=True)

In [5]:
input_ids = tokenizer.encode("Where did John marry? \\n John was born in New York.", return_tensors="pt")
res = model.generate(input_ids)

In [6]:
res

tensor([[    0,   126, 25453,     1]])

In [7]:
tokenizer.batch_decode(res, skip_special_tokens=True)

['new york']

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [10]:
input_ids = tokenizer('John was born in New York. Where did John marry?', return_tensors='pt').input_ids
labels = tokenizer('New York.', return_tensors='pt').input_ids
# the forward function automatically creates the correct decoder_input_ids
output = model(input_ids=input_ids, labels=labels)

In [11]:
output

Seq2SeqLMOutput(loss=tensor(2.8622, grad_fn=<NllLossBackward>), logits=tensor([[[-15.9840,  -7.6794, -12.9570,  ..., -42.6001, -42.5607, -42.5265],
         [-28.1204, -13.5563, -13.4772,  ..., -43.0448, -43.1281, -43.1451],
         [-17.1503,  -3.7817,  -7.1813,  ..., -37.5219, -37.4158, -37.5029],
         [-23.7658,  -2.1364,  -9.3628,  ..., -42.2548, -42.2119, -42.1693]]],
       grad_fn=<UnsafeViewBackward>), past_key_values=((tensor([[[[-0.4865, -2.3323, -1.1428,  ..., -2.5693, -1.7539, -0.5693],
          [ 0.1262,  0.4663,  1.2347,  ..., -1.0560, -0.4323,  1.9126],
          [ 0.6331, -0.3175, -0.0695,  ..., -0.8433, -0.1205,  0.0237],
          [-0.8118,  0.1542,  0.0627,  ..., -0.1274, -0.4982, -0.0704]],

         [[-0.9565, -2.3581, -0.2478,  ...,  1.1193,  0.7770,  0.5377],
          [-0.5764,  0.7417, -0.9517,  ...,  2.5385,  0.7979,  1.1393],
          [-0.7639,  0.6820, -0.4498,  ...,  1.5959,  1.5463,  1.2838],
          [-1.1496, -2.3143, -1.5079,  ...,  0.9651,  0.1

In [64]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

inputs = tokenizer("John was born in New York. Where did John marry?", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

In [44]:
torch.index_select(next_token_logits, 1, inputs['input_ids'].squeeze())


tensor([[-136.6187, -136.9380, -140.0671, -136.3882, -134.1165, -137.5126,
         -136.4332, -132.9195, -139.2103, -132.2246, -142.7845, -138.0391]],
       grad_fn=<IndexSelectBackward>)

In [48]:
for i in inputs['input_ids'].squeeze():
    print(tokenizer.decode(i))

John
 was
 born
 in
 New
 York
.
 Where
 did
 John
 marry
?


In [96]:
inputs['input_ids']

tensor([[ 7554,   373,  4642,   287,   968,  1971,    13,  6350,   750,  1757,
         12479,    30]])

In [52]:
prob_first_token = torch.index_select(next_token_logits, 1, inputs['input_ids'].squeeze())
torch.topk(prob_first_token, 10).indices.squeeze()

tensor([ 9,  7,  4,  3,  6,  0,  1,  5, 11,  8])

In [58]:
tokenizer.decode(inputs['input_ids'].squeeze()[9:10])

' John'

In [268]:
tokenizer.decode([7])

'('

In [277]:
def logprobs(input_ids, attention_mask):
    outputs = model(input_ids = input_ids, attention_mask = attention_mask)
    logits = outputs[0][:, -1, :]
    logprobs = F.log_softmax(logits, dim=-1)
    return torch.index_select(logprobs, 1, input_ids.squeeze()) * attention_mask

In [281]:
a = torch.tensor([[7554, 373, 4642, 50256, 50256]])
b = torch.tensor([[1.0, 1.0, 1.0, 0.0, 0.0]], dtype=torch.float)
outputs = model(input_ids = a, attention_mask = b)
logits = outputs[0][:, -1, :]
logprobs = F.log_softmax(logits, dim=-1)
torch.index_select(logprobs, 1, a.squeeze()) / b

tensor([[ -7.4898, -13.9429, -18.9858,     -inf,     -inf]],
       grad_fn=<DivBackward0>)

In [283]:
a.shape[0]

1

In [276]:
torch.index_select(logprobs, 1, a.squeeze())

tensor([[ -7.4898, -13.9429, -18.9858,  -7.8199,  -7.8199]],
       grad_fn=<IndexSelectBackward>)

In [247]:
R = 10
L = 20
K = 10

instance = "John was born in New York. Where did John marry?"
def candidate_spans(instance):
    inputs = tokenizer(instance, return_tensors="pt")
    logprob = torch.log(torch.zeros([R, L], dtype = torch.float32))
    logprob_first_token = logprobs(inputs['input_ids']).squeeze()
    first_token_index = torch.topk(logprob_first_token, R).indices.squeeze()
    
    for i, index in enumerate(first_token_index):
        for l in range(L):
            if index+l+1 < inputs['input_ids'].squeeze().shape[0]:
                if l == 0:
                    logprob[i][l] = logprob_first_token[index]
                tmp = inputs['input_ids'].squeeze()[index:index+l+1] # range does not include right index
                new_instance = torch.cat((inputs['input_ids'], tmp.view(1, -1)), dim=1)
                logprob_tmp = logprobs(new_instance).squeeze()
                logprob[i][l+1] = logprob_tmp[index+l+1] + logprob[i][l]
                
    topk_logprob = torch.topk(logprob.flatten(), K)
    topk_span_ids = topk_logprob.indices
    spans = []
    for i in topk_span_ids:
        start = first_token_index[int(i/L)]
        length = i % L
        spans.append([start, start+length+1])
    return torch.softmax(topk_logprob.values, dim = -1), spans

In [218]:
logprob_first_token = logprobs(inputs['input_ids']).squeeze()
torch.topk(logprob_first_token, R).indices.squeeze()

tensor([ 9,  7,  4,  3,  6,  0,  1,  5, 11,  8])

In [249]:
logprob, spans = candidate_spans(instance)
print(logprob)
print(spans)

tensor([0.4489, 0.2241, 0.1528, 0.0677, 0.0588, 0.0171, 0.0114, 0.0070, 0.0067,
        0.0055], grad_fn=<SoftmaxBackward>)
[[tensor(9), tensor(10)], [tensor(7), tensor(8)], [tensor(7), tensor(9)], [tensor(4), tensor(5)], [tensor(4), tensor(6)], [tensor(4), tensor(7)], [tensor(7), tensor(10)], [tensor(3), tensor(4)], [tensor(6), tensor(7)], [tensor(0), tensor(1)]]


In [240]:
topk_logprob = torch.topk(logprob.flatten(), K)

In [245]:
topk_logprob.values

tensor([-3.9387, -4.6335, -5.0161, -5.8306, -5.9705, -7.2063, -7.6091, -8.1023,
        -8.1472, -8.3328], grad_fn=<TopkBackward>)

In [246]:
torch.softmax(topk_logprob.values, dim = -1)

tensor([0.4489, 0.2241, 0.1528, 0.0677, 0.0588, 0.0171, 0.0114, 0.0070, 0.0067,
        0.0055], grad_fn=<SoftmaxBackward>)

In [236]:
for span in spans:
    print(tokenizer.decode(inputs['input_ids'].squeeze()[span[0]:span[1]]))

 John
 Where
 Where did
 New
 New York
 New York.
 Where did John
 in
.
John


In [229]:
spans

[tensor([1757]),
 tensor([ 1757, 12479]),
 tensor([6350]),
 tensor([6350,  750]),
 tensor([6350,  750, 1757]),
 tensor([ 6350,   750,  1757, 12479]),
 tensor([968]),
 tensor([ 968, 1971]),
 tensor([ 968, 1971,   13]),
 tensor([ 968, 1971,   13, 6350]),
 tensor([ 968, 1971,   13, 6350,  750]),
 tensor([ 968, 1971,   13, 6350,  750, 1757]),
 tensor([  968,  1971,    13,  6350,   750,  1757, 12479]),
 tensor([287]),
 tensor([287, 968]),
 tensor([ 287,  968, 1971]),
 tensor([ 287,  968, 1971,   13]),
 tensor([ 287,  968, 1971,   13, 6350]),
 tensor([ 287,  968, 1971,   13, 6350,  750]),
 tensor([ 287,  968, 1971,   13, 6350,  750, 1757]),
 tensor([  287,   968,  1971,    13,  6350,   750,  1757, 12479]),
 tensor([13]),
 tensor([  13, 6350]),
 tensor([  13, 6350,  750]),
 tensor([  13, 6350,  750, 1757]),
 tensor([   13,  6350,   750,  1757, 12479]),
 tensor([7554]),
 tensor([7554,  373]),
 tensor([7554,  373, 4642]),
 tensor([7554,  373, 4642,  287]),
 tensor([7554,  373, 4642,  287,  968]